In [79]:
import requests
import polars as pl
from dotenv import load_dotenv
import os
from pathlib import Path
from google.cloud import language_v2
from google import genai
from google.genai import types


In [80]:
PROJECT_PATH = "dstack/d-stack-home"
RAW_PATH = "raw_issues.parquet"
CLEANED_PATH = "cleaned_issues.parquet"
KEYWORDS_PATH = "keywords_config.txt"

load_dotenv()

# Access the API key
api_key = os.environ.get("API_KEY_GCP")
token_vertex = os.environ.get("ACCES_TOKEN_VERTEX")
PROJECT_ID = "project-8415b93b-4a16-4c2b-901"
LOCATION = "europe-west3"
print("API key loaded:", bool(api_key))
client = language_v2.LanguageServiceClient(client_options={"api_key": api_key})

keywords_path = Path(KEYWORDS_PATH)

LABELS = [
    line.strip()
    for line in keywords_path.read_text(encoding="utf-8").splitlines()
    if line.strip() and not line.lstrip().startswith("#")
]

API key loaded: True


In [9]:
def fetch_all_gitlab_issues(
    project_path: str,
    gitlab_base_url: str = "https://gitlab.opencode.de",
    per_page: int = 100,
    timeout: int = 30,
):
    """
    Fetch all issues from a public GitLab project.

    project_path: "namespace/project", e.g. "dstack/d-stack-home"
    returns: list of issue JSON objects
    """
    session = requests.Session()
    session.headers.update({"Accept": "application/json"})

    # URL-encode project path
    project_path_enc = project_path.replace("/", "%2F")
    base_api = f"{gitlab_base_url}/api/v4/projects/{project_path_enc}/issues"

    issues = []
    page = 1

    while True:
        resp = session.get(
            base_api,
            params={
                "state": "all",
                "per_page": per_page,
                "page": page,
                "order_by": "created_at",
                "sort": "asc",
            },
            timeout=timeout,
        )
        resp.raise_for_status()
        batch = resp.json()

        if not batch:
            break

        issues.extend(batch)
        page += 1

    return issues

def load_gitlab_issues(path: str) -> pl.DataFrame:
    # load from parquet file
    pass


In [10]:
# issues = fetch_all_gitlab_issues("dstack/d-stack-home")
# raw_df = pl.DataFrame(issues)
# raw_df.write_parquet(RAW_PATH)

In [11]:
raw_df = pl.read_parquet(RAW_PATH)

In [12]:
# some iid s to exclude since they predate the feedback process
iids_to_exclude = list(range(1,9))
columns_to_keep = ["iid", "title", "description", "state", "created_at", "updated_at", "closed_at", "author_id", "author_name", "author_state", "user_notes_count", "upvotes", "downvotes", "references"]
desc_to_exclude = ["", "test", "Test"]

In [13]:
def clean_issues_df(df: pl.DataFrame, columns_to_use: list, rows_to_exclude: list) -> pl.DataFrame:
    df = df.with_columns([
        pl.col("author").struct.field("id").alias("author_id"),
        pl.col("author").struct.field("name").alias("author_name"),
        pl.col("author").struct.field("state").alias("author_state"),
    ])
    df = df.with_columns([
        pl.col("created_at").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.fZ").alias("created_at"),
        pl.col("updated_at").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.fZ").alias("updated_at"),
        pl.col("closed_at").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.fZ").alias("closed_at"),
    ])
    df = df.filter(~pl.col("iid").is_in(rows_to_exclude))
    df = df.unique(subset=["title", "description"])
    return df.select(columns_to_use)
    

In [14]:
def enrich_issues_df(df: pl.DataFrame, desc_to_exclude: list) -> pl.DataFrame:
    # clean description
    df = df.with_columns(
        desc_clean = pl.col("description").str.replace_all("**Feedback:** <br>", "", literal=True)
    )
    # get more insights on where issue comes from
    df = df.with_columns(
        is_from_form = pl.col("title").str.starts_with("Feedback für die Seite"),
        form_page = (
            pl.when(pl.col("title").str.starts_with("Feedback für die Seite"))
            .then(
                pl.col("title")
                .str.replace("^Feedback für die Seite", "")
                .str.strip_chars()
            )
            .otherwise(pl.lit("Via OpenCode"))
        )
    )
    df = df.filter(~pl.col("desc_clean").is_in(desc_to_exclude))
    return df

In [15]:
df_clean = clean_issues_df(raw_df, columns_to_keep, iids_to_exclude)

In [16]:
df = enrich_issues_df(df_clean, desc_to_exclude)

In [17]:
df.sort(by="upvotes", descending=True).head()

iid,title,description,state,created_at,updated_at,closed_at,author_id,author_name,author_state,user_notes_count,upvotes,downvotes,references,desc_clean,is_from_form,form_page
i64,str,str,str,datetime[μs],datetime[μs],datetime[μs],i64,str,str,i64,i64,i64,struct[3],str,bool,str
343,"""Domäne Sicherheit, Fehlende Th…","""Die Domäne Sicherheit könnte n…","""opened""",2025-11-21 15:13:17.800,2025-11-21 15:13:17.800,null,1654,"""Jürgen Bilberger""","""active""",0,11,0,"{""#343"",""#343"",""dstack/d-stack-home#343""}","""Die Domäne Sicherheit könnte n…",false,"""Via OpenCode"""
278,"""OSBA: Digitale Souveränität""","""Digitale Souveränität ist eine…","""opened""",2025-11-12 10:28:29.695,2025-11-13 14:54:50.060,null,12277,"""Burkhard Noltensmeier""","""active""",1,9,0,"{""#278"",""#278"",""dstack/d-stack-home#278""}","""Digitale Souveränität ist eine…",false,"""Via OpenCode"""
184,"""Ganzheitliche Standardisierung…","""**Ganzheitliche Standardisieru…","""opened""",2025-10-21 15:16:50.085,2025-12-01 08:49:58.195,null,12867,"""Jörg Friebe""","""active""",2,8,0,"{""#184"",""#184"",""dstack/d-stack-home#184""}","""**Ganzheitliche Standardisieru…",false,"""Via OpenCode"""
341,"""Domäne Integration, Observabil…","""Je nach Scope von ""Integration…","""opened""",2025-11-21 13:25:57.456,2025-11-21 13:25:57.456,null,13828,"""René Zarwel""","""active""",0,8,0,"{""#341"",""#341"",""dstack/d-stack-home#341""}","""Je nach Scope von ""Integration…",false,"""Via OpenCode"""
327,"""Domäne Inbetriebnahme, Empfehl…","""Die Auswahl der Tools für die …","""opened""",2025-11-21 09:12:58.691,2025-11-24 12:22:32.449,null,26,"""Dirk Gernhardt""","""active""",1,8,0,"{""#327"",""#327"",""dstack/d-stack-home#327""}","""Die Auswahl der Tools für die …",false,"""Via OpenCode"""


In [33]:
daily_counts_df = (
    df
    .group_by(
        pl.col("created_at").dt.date().alias("date")
    )
    .len()
    .rename({"len": "count"})
    .sort("date")
)

chart = (
    daily_counts_df
    .plot.line(
        x="date",       # Explicitly set the X-axis
        y="count",      # Explicitly set the Y-axis
        # title="Daily Activity" # Set a title
    )
    .properties(width=500)
    # Altair chart configurations can be chained
    .configure_axisY(title="Count of Records")
)

# chart

## Some analysis of the feedback

- distinct feedbackers
- form-generated vs manual issues (anonymous vs non-anonymous feedbackers)
- Issues per page 
- length of issues
- temporal analysis - when were issues commited?
- sentiment analysis
- keyword / label counts: first get the distinct labels, clean them, manually add some more, keyword search them in titles and descriptions
- correlations sentiments and labels / topics
- correlations between upvotes / downvotes and topics
- top comments
- Amount of non-content tickets ("Test" etc.)

## Sentiment analysis

In [ ]:
def get_sentiment_score(text_content: str) -> float:
    """
    Analyzes the sentiment of a single string and returns only the float score.
    Returns 0.0 if the input is None or the API call fails.
    """
    if text_content is None:
        return 0.0

    try:
        # Create the document object
        document = language_v2.Document(
            content=text_content, 
            type_=language_v2.Document.Type.PLAIN_TEXT
        )

        # Call the API
        response = client.analyze_sentiment(document=document)
        
        # Return the sentiment score (-1.0 to +1.0)
        return response.document_sentiment.score

    except Exception as e:
        print(f"Error processing text: {text_content[:20]}... Error: {e}")
        return 0.0 # Return a neutral score on error



In [ ]:
# df_with_sentiment = df.with_columns(
#     pl.col("desc_clean")
#       .map_elements(get_sentiment_score, return_dtype=pl.Float64)
#       .alias("sentiment")
# )

# df_with_sentiment.write_parquet("enriched_df.parquet")

In [28]:
df_with_sentiment = pl.read_parquet("enriched_df.parquet")

In [29]:
df_with_sentiment.sort(by="sentiment").sample(10).select("desc_clean", "sentiment").rows()

[('**Für digitale Verwaltung bedarf es EINES Nachschlageortes und EINES Ortes der Wahrheit für Themen der digitalen Verwaltung**\n\nInformation:\nDas BMDS strebt an, die bisherigen Glossare im Umfeld\n\n1. https://docs.fitko.de/fit/fit-sb/glossary/ \n2. https://docs.fitko.de/fit/glossary/ \n3. https://docs.fitko.de/fit-connect/docs/glossary/ \n\nzu konsolidieren und auch für den Deutschland-Stack zu nutzen.\n\nAls Anforderung zumindest für den Deutschland-Stack, aber **idealerweise auch für digitale Verwaltung insgesamt ** ist:\n1. EINE verbindliche Begriffsdefinition\n2. EIN Ort der Wahrheit für Fach- und technische Fragen\n3. Hierfür bedarf es einer Governance, die frei von Hierarchie und ohne Wasserfall funktioneriert.\n4. Ähnlich wie für die Standardfabrik empfehle ich ein knappes verbindliches Regelwerk, das orientiert an den Vorgaben für open standard angelegt ist.\n\nErläuterung \nEin Nachschlagewerk sollte - gerade um in der föderalen Verwaltungsstruktur Deutschlands zu wirken 

# Labeling

In [ ]:
df_with_sentiment

In [69]:
SYSTEM_INSTRUCTION = (
    "Du bekommst GitLab-Issues aus dem Deutschland-Stack-Konsultationsverfahren. "
    "Du bist ein Klassifizierungs-Experte. Deine Aufgabe ist es, die Beschreibung zu analysieren "
    "und sie anhand der Labels in der Liste zu klassifizieren. "
    "Nutze NUR die zur Verfügung gestellten Labels. Erfinde keine neuen Labels! "
    "Stelle das Ergebnis als Komma-separierten String zur Verfügung. "
    "Der String enthält NUR die von dir vergebenen Labels (eins oder bis zu 5)."
    "Wenn kein Label passt, nutze das Label Unklar"
)


In [82]:
client = genai.Client(vertexai=True)

MODEL = "gemini-2.0-flash"

In [83]:
def validate_labels(labels_str: str, allowed_labels: list[str]) -> list[str]:
    """
    Filtert die durch Komma getrennten Labels und entfernt alle,
    die nicht in allowed_labels sind.
    """
    labels = [label.strip() for label in labels_str.split(",")]
    return [label for label in labels if label in allowed_labels]


In [84]:
def classify_issue_multilabel(issue_text: str, labels: list[str]) -> str:
    user_prompt = f"""
Labels:
{", ".join(labels)}

Issue:
{issue_text}
"""

    response = client.models.generate_content(
        model=MODEL,
        contents=[
            types.Content(
                role="user",
                parts=[types.Part(text=SYSTEM_INSTRUCTION + "\n\n" + user_prompt)],
            ),
        ],
    )

    return response.text.strip()


In [87]:
test = df_with_sentiment.sample(3).with_columns(
    pl.col("desc_clean")
    .map_elements(
        lambda text: validate_labels(
            classify_issue_multilabel(text, LABELS),
            LABELS,
        ),
        return_dtype=pl.List(pl.Utf8),
    )
    .alias("labels")
)

In [88]:
test.select("desc_clean", "sentiment", "labels").rows()

[('Bei der Inbetriebnahme sind normalerweise mehrere Automatisierungstools im Einsatz. Für den Zugriff eines Tools auf ein anderes oder zur Installation einer Anwendung in die Betriebsumgebung werden üblicherweise Tokens benötigt. Diese Tokens müssen konfiguriert und ausgetauscht werden. Tokens kommen auch bei Zugriffen einer Anwendung auf Schnittstellen einer anderen Anwendung zum Einsatz. Zur Verwaltung der Tokens werden ist es insbesondere bei einer hohen Anzahl von Tokens sinnvoll, Secretmanagement Tools wie z.B. die folgenden einzusetzen:\n\n* OpenBao (https://openbao.org/, Mozilla Public License, version 2.0)\n* Infisical (https://infisical.com/, MIT expat license)',
  0.007000000216066837,
  ['Betrieb', 'Infrastruktur', 'Sicherheit', 'Toolvorschlag']),
 ('Die Inbetriebnahme sollte so gut wie möglich automatisiert ablaufen. Vorherige Prüfungen wie die Durchführung automatisierter Tests, Einhaltung von Programmierrichtlinien oder Security Checks sind dabei Standard. Im folgenden e